# Chapter 37: Compile, Eval, and Exec

This notebook covers Python's built-in functions for dynamic code execution: `compile`, `eval`, and `exec`. You will learn how to compile source code and ASTs into code objects, evaluate expressions with custom namespaces, execute statements dynamically, and inspect code object attributes.

## Key Concepts
- **`eval`**: Evaluates a single expression and returns the result
- **`exec`**: Executes arbitrary statements (no return value)
- **`compile`**: Creates a reusable code object from source or an AST
- **Code objects**: Low-level compiled bytecode with inspection attributes
- **`__code__`**: Access a function's code object for introspection

## Section 1: Evaluating Expressions with `eval`

`eval` takes a string containing a single Python expression, evaluates it, and returns the result. It can optionally accept global and local namespace dictionaries.

In [ ]:
# Basic eval usage
result: int = eval("2 + 3")  # noqa: S307
print(f"2 + 3 = {result}")
print(f"Result is 5: {result == 5}")

# eval works with any expression
list_result: list[int] = eval("[i ** 2 for i in range(5)]")  # noqa: S307
print(f"\nSquares: {list_result}")

string_result: str = eval("'hello'.upper()")  # noqa: S307
print(f"Uppercase: {string_result}")

In [ ]:
# eval with a custom namespace
ns: dict[str, int] = {"x": 10, "y": 20}
result: int = eval("x + y", ns)  # noqa: S307

print(f"x + y = {result}")
print(f"Result is 30: {result == 30}")

In [ ]:
# eval with separate global and local namespaces
global_ns: dict[str, int] = {"base": 100}
local_ns: dict[str, int] = {"offset": 42}

result: int = eval("base + offset", global_ns, local_ns)  # noqa: S307
print(f"base + offset = {result}")

# Using built-in functions through eval
result2: int = eval("max(10, 20, 30)", {"__builtins__": __builtins__})  # noqa: S307
print(f"max(10, 20, 30) = {result2}")

# Restricting builtins for safety
safe_ns: dict[str, object] = {"__builtins__": {}, "x": 5}
result3: int = eval("x * 2", safe_ns)  # noqa: S307
print(f"x * 2 (restricted) = {result3}")

## Section 2: Executing Statements with `exec`

`exec` runs arbitrary Python statements. Unlike `eval`, it does not return a value. Variables created during execution are placed into the provided namespace dictionary.

In [ ]:
# exec runs statements and populates the namespace
ns: dict[str, object] = {}
exec("result = [i**2 for i in range(5)]", ns)  # noqa: S102

print(f"result: {ns['result']}")
print(f"Expected: {[0, 1, 4, 9, 16]}")
print(f"Match: {ns['result'] == [0, 1, 4, 9, 16]}")

In [ ]:
# exec can define functions and classes
ns: dict[str, object] = {}
exec("""  # noqa: S102
def greet(name):
    return f"Hello, {name}!"

message = greet("World")
""", ns)

print(f"message: {ns['message']}")
print(f"greet is callable: {callable(ns['greet'])}")

In [ ]:
# exec with pre-populated namespace
ns: dict[str, object] = {"data": [3, 1, 4, 1, 5, 9, 2, 6]}
exec("""  # noqa: S102
sorted_data = sorted(data)
total = sum(data)
average = total / len(data)
""", ns)

print(f"Sorted: {ns['sorted_data']}")
print(f"Total: {ns['total']}")
print(f"Average: {ns['average']}")

## Section 3: Compiling Source Code

`compile` creates a code object from a source string or an AST. The code object can then be passed to `eval` or `exec`. The `mode` parameter controls what kind of code is expected:
- `"exec"`: A module (sequence of statements)
- `"eval"`: A single expression
- `"single"`: A single interactive statement

In [ ]:
import types

# Compile source code to a code object
code: types.CodeType = compile("x = 42", "<test>", "exec")

print(f"Type: {type(code).__name__}")
print(f"Is CodeType: {isinstance(code, types.CodeType)}")

# Execute the compiled code
ns: dict[str, object] = {}
exec(code, ns)  # noqa: S102
print(f"\nx = {ns['x']}")
print(f"x == 42: {ns['x'] == 42}")

In [ ]:
# Compile in eval mode for expressions
expr_code = compile("2 ** 10", "<expr>", "eval")
result: int = eval(expr_code)  # noqa: S307
print(f"2 ** 10 = {result}")

# Compile once, execute multiple times with different namespaces
formula = compile("a * b + c", "<formula>", "eval")

inputs: list[dict[str, int]] = [
    {"a": 2, "b": 3, "c": 4},
    {"a": 5, "b": 6, "c": 7},
    {"a": 10, "b": 10, "c": 10},
]

print("\nEvaluating 'a * b + c':")
for ns in inputs:
    result = eval(formula, ns)  # noqa: S307
    print(f"  a={ns['a']}, b={ns['b']}, c={ns['c']} -> {result}")

In [ ]:
import ast
import types

# Compile from an AST instead of source text
tree: ast.Module = ast.parse("y = 100 + 200")
code: types.CodeType = compile(tree, "<ast>", "exec")

print(f"Compiled from AST: {isinstance(code, types.CodeType)}")

ns: dict[str, object] = {}
exec(code, ns)  # noqa: S102
print(f"y = {ns['y']}")

## Section 4: Code Object Attributes

Code objects contain compiled bytecode along with metadata. You can access a function's code object through its `__code__` attribute and inspect properties like the function name, argument count, and variable names.

In [ ]:
# Inspect a function's code object
def example(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


code = example.__code__

print(f"co_name: {code.co_name}")
print(f"co_argcount: {code.co_argcount}")
print(f"co_varnames: {code.co_varnames}")
print(f"\n'a' in co_varnames: {'a' in code.co_varnames}")
print(f"'b' in co_varnames: {'b' in code.co_varnames}")

In [ ]:
# More code object attributes
def calculate(x: int, y: int, z: int) -> int:
    """Perform a calculation with three values."""
    temp: int = x + y
    result: int = temp * z
    return result


code = calculate.__code__

print(f"co_name: {code.co_name}")
print(f"co_argcount: {code.co_argcount}")
print(f"co_varnames: {code.co_varnames}")
print(f"co_filename: {code.co_filename}")
print(f"co_nlocals: {code.co_nlocals}")
print(f"co_consts: {code.co_consts}")

In [ ]:
# Compare code objects from different functions
def add(a: int, b: int) -> int:
    return a + b


def greet(name: str, greeting: str, punctuation: str) -> str:
    message: str = greeting + ", " + name + punctuation
    return message


functions = [add, greet]
for func in functions:
    code = func.__code__
    print(f"{code.co_name}:")
    print(f"  Arguments: {code.co_argcount}")
    print(f"  Local vars: {code.co_varnames}")
    print(f"  Num locals: {code.co_nlocals}")
    print()

## Section 5: Combining AST, Compile, and Exec

A common workflow is to parse source into an AST, transform it, compile the modified AST, and execute it. This enables powerful metaprogramming patterns.

In [ ]:
import ast


class AddPrintTracing(ast.NodeTransformer):
    """Wraps each assignment with a print statement showing the value."""

    def visit_Assign(self, node: ast.Assign) -> list[ast.stmt]:
        # Keep the original assignment
        # Add a print call after it
        target = node.targets[0]
        if isinstance(target, ast.Name):
            print_call: ast.Expr = ast.Expr(
                value=ast.Call(
                    func=ast.Name(id="print", ctx=ast.Load()),
                    args=[
                        ast.Constant(value=f"  {target.id} ="),
                        ast.Name(id=target.id, ctx=ast.Load()),
                    ],
                    keywords=[],
                )
            )
            return [node, print_call]
        return [node]


source: str = """
x = 5
y = 10
z = x + y
"""

tree: ast.Module = ast.parse(source)
traced: ast.Module = AddPrintTracing().visit(tree)
ast.fix_missing_locations(traced)

code = compile(traced, "<traced>", "exec")
print("Running traced code:")
ns: dict[str, object] = {}
exec(code, ns)  # noqa: S102

In [ ]:
import ast
import types

# Full pipeline: parse -> analyze -> transform -> compile -> execute
source: str = "price = 10"

# Step 1: Parse
tree: ast.Module = ast.parse(source)
print(f"Step 1 - Parsed: {ast.dump(tree)}")

# Step 2: Analyze
node_count: int = sum(1 for _ in ast.walk(tree))
print(f"Step 2 - Node count: {node_count}")


# Step 3: Transform (triple all constants)
class TripleConstants(ast.NodeTransformer):
    def visit_Constant(self, node: ast.Constant) -> ast.Constant:
        if isinstance(node.value, int):
            return ast.Constant(value=node.value * 3)
        return node


new_tree: ast.Module = TripleConstants().visit(tree)
ast.fix_missing_locations(new_tree)
print(f"Step 3 - Transformed: {ast.dump(new_tree)}")

# Step 4: Compile
code: types.CodeType = compile(new_tree, "<pipeline>", "exec")
print(f"Step 4 - Compiled: {isinstance(code, types.CodeType)}")

# Step 5: Execute
ns: dict[str, object] = {}
exec(code, ns)  # noqa: S102
print(f"Step 5 - price = {ns['price']} (original was 10, tripled to 30)")

## Section 6: Security Considerations

Both `eval` and `exec` execute arbitrary code, which makes them dangerous with untrusted input. Use `ast.literal_eval` for safe parsing of data, and always restrict namespaces when dynamic evaluation is necessary.

In [ ]:
import ast

# Safe approach: use ast.literal_eval for data parsing
safe_inputs: list[str] = ["42", "[1, 2, 3]", "{'key': 'value'}"]

print("Safe parsing with ast.literal_eval:")
for inp in safe_inputs:
    result = ast.literal_eval(inp)
    print(f"  {inp!r} -> {result} ({type(result).__name__})")

# Restricted eval: empty builtins prevents access to dangerous functions
print("\nRestricted eval:")
restricted_ns: dict[str, object] = {"__builtins__": {}, "x": 5, "y": 10}
result: int = eval("x + y", restricted_ns)  # noqa: S307
print(f"  x + y = {result} (with restricted builtins)")

## Summary

### eval
- **`eval(expression, globals, locals)`**: Evaluates a single expression and returns the result
- Accepts an optional namespace for variable resolution
- Can evaluate pre-compiled code objects

### exec
- **`exec(code, globals, locals)`**: Executes statements (assignments, definitions, loops)
- Does not return a value; results are stored in the namespace dict
- Can execute strings, code objects, or compiled ASTs

### compile
- **`compile(source, filename, mode)`**: Creates a reusable code object
- Modes: `"exec"` for statements, `"eval"` for expressions, `"single"` for interactive
- Accepts source strings or `ast.Module` / `ast.Expression` trees

### Code Objects
- **`function.__code__`**: Access a function's compiled code object
- **`co_name`**: The function name
- **`co_argcount`**: Number of positional arguments
- **`co_varnames`**: Tuple of local variable names (arguments first)
- **`co_nlocals`**: Total number of local variables
- **`co_consts`**: Tuple of constants used in the function

### Security
- Never use `eval` or `exec` with untrusted input
- Use `ast.literal_eval` for safe parsing of literal data
- Restrict `__builtins__` in namespaces to limit available functions